# **<span style="color: black; font-size:1em;">PrOMMiS LCA Integration: Environmental Life Cycle Assessment Using PrOMMiS Modeling Results</span>**

## <span style="color:black; font-weight:bold"> Introduction </span> 

<span style = "color:black">
This document demonstrates the application of Life Cycle Assessment (LCA) methodology to the optimized results from the PrOMMiS model.  
This Jupyter Notebook presents an example LCA application to the West Kentucky No.13 Coal Refuse Flowsheet optimized in the PrOMMiS model.

**Useful notes, definitions, and links:**
- **PrOMMiS model:** Process Optimization and Modeling for Minerals Sustainability initiative, led by the U.S. Department of Energy (DOE), specifically under NETL (National Energy Technology Laboratory).
- **PrOMMiS public GitHub Repository:** [https://github.com/prommis/prommis/tree/main](https://github.com/prommis/prommis/tree/main)
- **UKy Flowsheet Code:** [https://github.com/prommis/prommis/blob/main/src/prommis/uky/uky_flowsheet.py](https://github.com/prommis/prommis/blob/main/src/prommis/uky/uky_flowsheet.py)

</span>

<div style="text-align: center;">
    <img src="images/uky_flowsheet.png" width="1000"/>
</div>

## <span style="color:black; font-weight:bold"> Step 1: Import the necessary tools </span> 

In [ ]:
# import main libraries

import pandas as pd

from netlolca.NetlOlca import NetlOlca

import prommis.uky.uky_flowsheet as uky # module to run the PrOMMiS model
import src as lca_prommis


## <span style="color:black; font-weight:bold"> Step 2: Run PrOMMiS Optimization Model for the UKy flowsheet and extract model results </span> 

In [ ]:
# Run PrOMMiS model 
m, results = uky.main() 

# Extract LCA data
prommis_data = lca_prommis.data_lca.get_lca_df(m, )
prommis_data.to_csv("lca_df.csv")

# TODO: Leave a note for the user with a link to the prommis repo

Initialization Order
fs.leach_solid_feed
fs.leach
fs.sl_sep1
fs.leach_mixer
fs.sc_circuit_purge
fs.sl_sep2
fs.precip_sep
fs.translator_precip_sep_to_purge
fs.precip_purge
INFO Starting Sequential Decomposition
INFO Starting first pass run of network
2025-08-22 15:41:09 [INFO] idaes.prommis.uky.uky_flowsheet: Initializing fs.leach_solid_feed
INFO Initializing fs.leach_solid_feed
2025-08-22 15:41:09 [INFO] idaes.prommis.uky.uky_flowsheet: Initializing fs.leach_liquid_feed
INFO Initializing fs.leach_liquid_feed
2025-08-22 15:41:12 [INFO] idaes.prommis.uky.uky_flowsheet: Initializing fs.solex_rougher_load
INFO Initializing fs.solex_rougher_load
2025-08-22 15:41:13 [INFO] idaes.init.fs.solex_rougher_load.mscontactor: Stream Initialization Completed.
2025-08-22 15:41:14 [INFO] idaes.init.fs.solex_rougher_load.mscontactor: Initialization Completed, optimal - <undefined>
2025-08-22 15:41:14 [INFO] idaes.prommis.uky.uky_flowsheet: Initializing fs.rougher_org_make_up
INFO Initializing fs.rougher

## <span style="color:black; font-weight:bold"> Step 3: Review PrOMMiS Results </span> 

In [3]:
prommis_data.head()

,Flow,Source,In/Out,Category,Value 1,Unit 1,Value 2,Unit 2
0,Inerts,Solid Feed,In,Solid Input,22.68,kg/hr,0.6952,mass fraction
1,Scandium Oxide,Solid Feed,In,Solid Input,22.68,kg/hr,0.000028,mass fraction
2,Yttrium Oxide,Solid Feed,In,Solid Input,22.68,kg/hr,0.000033,mass fraction
3,Lanthanum Oxide,Solid Feed,In,Solid Input,22.68,kg/hr,0.000068,mass fraction
4,Cerium Oxide,Solid Feed,In,Solid Input,22.68,kg/hr,0.000156,mass fraction


## <span style="color:black; font-weight:bold"> Step 4: Organize, Categorize, and Covert PrOMMiS Results to LCA Relevant Flows </span> 

In [4]:
# Hours is the time period for the conversion
# mol_to_kg is a boolean that indicates whether to convert moles to kg
# water_unit is the unit of water (m3 or L recommended, kg is also an option)
df = lca_prommis.convert_lca.convert_flows_to_lca_units(prommis_data, hours=1, mol_to_kg=True, water_unit='m3')
df.to_csv('lca_df_converted.csv', index=False)

# Finalize the LCA data
df = pd.read_csv('lca_df_converted.csv')
df = lca_prommis.final_lca.main()
df.to_csv('lca_df_finalized.csv', index=False)

# This will create a new datafrane and csv file that contains the initial prommis_data df with two additional columns:
# 'LCA Amount' and 'LCA Unit'

WARNING Dynamically registering the following numeric type:
    pymatgen.core.units.FloatWithUnit
Dynamic registration is supported for convenience, but there are known
limitations to this approach.  We recommend explicitly registering
numeric types using RegisterNumericType() or RegisterIntegerType().
Applied functional unit conversion with scaling factor: 1.5965288953841764e-05
Reference flow: 99.85% REO Product from Roaster Product
INFO Returning C:\Users\franc\AppData\Local\fedelemflowlist\FedElemFlowListMaster_v1.3.0_a79846d.parquet
Summary:
  total_flows: 17
  input_flows: 10
  output_flows: 7
  reference_products: 1
  unique_flow_types: 9
  total_lca_amount: 26057242.590626873

Flow Type Breakdown:
  Chemicals: 6
  Emissions to air: 4
  Solid Input: 1
  Solid Output: 1
  Electricity: 1
  Heat: 1
  Solid Waste: 1
  Wastewater: 1
  Water: 1


## <span style="color:black; font-weight:bold"> Step 5: Normalize Flows to the Selected Functional Unit and Review Final LCA Flows </span> 

In [5]:
# The below code will normalize the converted prommis data to a functional unit
# Note: This code is developed for the UKy flowsheet and the functional unit is automatically set to 1 kg of REO (combination of all REEs)
# TODO: This code should be developed for the other flowsheets. To achieve this, the user should be able to specify the desired functional unit
df = pd.read_csv('lca_df_converted.csv')

# Run the merge_flows function for the feed
REO_list = [
    "Yttrium Oxide",
    "Lanthanum Oxide",
    "Cerium Oxide",
    "Praseodymium Oxide",
    "Neodymium Oxide",
    "Samarium Oxide",
    "Gadolinium Oxide",
    "Dysprosium Oxide",
]
df = lca_prommis.final_lca.merge_flows(df, merge_source='Solid Feed', new_flow_name='374 ppm REO Feed', value_2_merge=REO_list)
# This 374 ppm value is directly calculated from the flowsheet. The original study actually used 357 ppm as the feed concentration.

# Run the merge_flows function for the product
df = lca_prommis.final_lca.merge_flows(df, merge_source='Roaster Product', new_flow_name='99.85% REO Product')

# Run the merge_flows function for the liquid waste flows
df = lca_prommis.final_lca.merge_flows(df, merge_source='Wastewater', new_flow_name='Wastewater', merge_column='Category') 
# Note: some of these streams are organic waste, but they're treated as wastewater

# Run the merge_flows function for the solid waste flows
df = lca_prommis.final_lca.merge_flows(df, merge_source='Solid Waste', new_flow_name='Solid Waste', merge_column='Category') 

# Run the finalize_df function
try:
    finalized_df = lca_prommis.final_lca.finalize_df(
        df=df, 
        reference_flow='99.85% REO Product', 
        reference_source='Roaster Product',
        water_type='raw fresh water'
    )
    
    # Get summary
    summary = lca_prommis.final_lca.get_finalize_summary(finalized_df)
    print("Summary:")
    for key, value in summary.items():
        if key != 'flow_type_breakdown':
            print(f"  {key}: {value}")
    
    print("\nFlow Type Breakdown:")
    for flow_type, count in summary['flow_type_breakdown'].items():
        print(f"  {flow_type}: {count}")
        
except Exception as e:
    print(f"Error during finalization: {e}")

finalized_df.to_csv('lca_df_finalized.csv', index=False)

Applied functional unit conversion with scaling factor: 1.5965288953841764e-05
Reference flow: 99.85% REO Product from Roaster Product
INFO Returning C:\Users\franc\AppData\Local\fedelemflowlist\FedElemFlowListMaster_v1.3.0_a79846d.parquet


Summary:
  total_flows: 17
  input_flows: 10
  output_flows: 7
  reference_products: 1
  unique_flow_types: 9
  total_lca_amount: 26057242.590626873

Flow Type Breakdown:
  Chemicals: 6
  Emissions to air: 4
  Solid Input: 1
  Solid Output: 1
  Electricity: 1
  Heat: 1
  Solid Waste: 1
  Wastewater: 1
  Water: 1


## <span style="color:black; font-weight:bold"> Step 6: Connect to openLCA </span> 

#### <span style="color:black; font-weight:bold"> Notes on modeling this process using openLCA </span> 

Before running the following code:
- Open the software openLCA
- Open an existing database or create a new one
- Import libraries with existing processes
- Use the IPC server from the Developer Tools
- Run the server on port 8080 

In [6]:
netl = NetlOlca()
netl.connect()
netl.read()

INFO Connected on http://localhost:8080
INFO Read UUIDs from IPC connection.


#### <span style="color:black; font-weight:bold"> Define dataframe to be used </span> 

<span style="color:black"> 

The dataframe should have the following columns: 

| Flow_Name | LCA_Unit | LCA_Amount | Is_Input | Reference_Product | Flow_Type | Description |

</span>

In [7]:
# Save the LCA data produced in previous steps in a dataframe called df
df = pd.read_csv('lca_df_finalized.csv')


#### <span style="color:black; font-weight:bold"> Enter Process Information </span> 

##### <span style="color:black; font-weight:bold"> Unit Process Name </span> 

In [8]:
process_name = "Test Process 1"  # Replace with the actual process name you want to use

##### <span style="color:black; font-weight:bold"> Unit Process Description </span> 

In [9]:
process_description = " Example Process Description"  # Replace with the actual process description you want to use

##### <span style="color:black; font-weight:bold"> Create process with exchanges </span> 

<span style="color:black">  

The functioning features include:
* Create a new empty process
* Create an exchange of the reference flow

Intermediate functions/features that have been developed include:
* Search for a flow by keyword
* Search for processes that produce a given flow
    * This feature is being optimized to minimize user waiting time
* Create exchanges for elementary flows given their uuid, amount, and unit

The features under development:
* Create exchange for a given flow and process IDs
* Develop an interactive user interface, allowing the user to create a new process in openLCA
</span>

In [ ]:
process = lca_prommis.create_lca.create_new_process(netl,df,process_name,process_description)
# TODO: give option for the user to enter another keyword when searching for a flow
# NOTE: For future discussion - is it better to select the flow then the provider (current status) or pick provider first then select relevant flows?